In [ ]:
import asyncio
from pyzeebe import ZeebeWorker, Job, create_camunda_cloud_channel, ZeebeClient

In [ ]:
ClusterURL = "fc43de37-01a5-41a0-a9d8-51d621169f35.dsm-1.zeebe.camunda.io:443"
ClientID = "UPPG5KY-b6ZRN1Yf_iuXG87hucOa286k"
ClientSecret = "J~pZPr6LZ6NxBa2Qwr7DVd8YwZahW35v8RIAMsi~h~VThu0.76GzL73sLB37CR2r"
ClusterID = "fc43de37-01a5-41a0-a9d8-51d621169f35"
Region = "dsm-1"

In [ ]:
channel = create_camunda_cloud_channel(client_id=ClientID, 
                                       client_secret=ClientSecret, 
                                       cluster_id=ClusterID, 
                                       region=Region)

In [ ]:
# Create a zeebe client and worker
client = ZeebeClient(channel)
worker = ZeebeWorker(channel)

In [ ]:
process_key = await client.run_process(bpmn_process_id='Antragssteller1', variables={})

In [ ]:
@worker.task(task_type="AntragStellenType")
async def AntragStellen(Alter: float, Einkommen: float, EinkommenMutter: float, EinkommenVater: float):

    corr_key_a="Message_08c9cbl"
    await client.publish_message(name=corr_key_a, correlation_key=corr_key_a, variables={"Alter": Alter,
        "Einkommen": Einkommen, "EinkommenMutter": EinkommenMutter, "EinkommenVater": EinkommenVater})
    
    return {}

@worker.task(task_type="AntragErneutStellenType")
async def AntragErneutStellen(Alter: float, Einkommen: float, EinkommenMutter: float, EinkommenVater: float):

    corr_key_b="Message_0acsdfj"
    await client.publish_message(name=corr_key_b, correlation_key=corr_key_b, variables={"Alter": Alter,
        "Einkommen": Einkommen, "EinkommenMutter": EinkommenMutter, "EinkommenVater": EinkommenVater})
    
    return {}

@worker.task(task_type="BitteVersendenType")
async def BitteVersenden():

    corr_key_c="Message_207673v"
    await client.publish_message(name=corr_key_c, correlation_key=corr_key_c,
          variables={"message": "Bitte Daten neu eingeben!"})
    
    return {}

@worker.task(task_type="AntragPruefenType")
async def AntragPruefen(Alter: float, Einkommen: float, EinkommenMutter: float, EinkommenVater: float):

    if Alter == -1:
        return {"Ergebnis": 0}
    if Einkommen == -1:
        return {"Ergebnis": 0}
    if EinkommenMutter == -1:
        return {"Ergebnis": 0}
    if EinkommenVater == -1:
        return {"Ergebnis": 0}
    
    return {"Ergebnis": 1}

@worker.task(task_type="ErgebnisUebermittelnType")
async def ErgebnisUebermitteln(BafoegErhoehung: float):

    corr_key_d="Message_1nv92jo"
    await client.publish_message(name=corr_key_d, correlation_key=corr_key_d, variables={"BafoegErhoehung": BafoegErhoehung})
    return {}

await worker.work()